### Forcasting Data processing

In [12]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import os
import glob
import csv
%matplotlib inline 

<b> 1. Combining all the csv files from January to November 2020 </b>

Here below, 11 csv files are combined, with a total of 218735 rows × 13 columns. The data set starts the 1st january and ands the 27th November 2020

In [13]:
#data path specification with all the desired csv data
data_path = r'C:\Users\Caleb\OneDrive - University of Edinburgh\Methodology & Consulting\Group Project\forecasting\data'
os.chdir(data_path)

#the following function combines the csv files in the desired folder for various time periods
#def csv_processing(data_path):

#combining all the csv files: matching the extension
extension_file = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension_file))]

#combine all files in the list
data2020_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv names 'data2020'
data2020_csv.to_csv( "data2020.csv", index=False, encoding='utf-8-sig')

#Loading the new csv file 
path = r'data2020.csv'
data2020 = pd.read_csv(path)


<b>2. Demand and Supply data per each station and hour </b>

In [14]:
data2020.head(3)

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,2020-01-01 00:11:49.789000+00:00,2020-01-01 00:32:37.402000+00:00,1247,1738,Wester Coates Terrace,Junction with Roseburn Terrace,55.945648,-3.231847,1725,Edinburgh Zoo,"Corstorphine Road, Edinburgh",55.942115,-3.269287
1,2020-01-01 00:15:47.185000+00:00,2020-01-01 00:24:48.541000+00:00,541,1024,Meadow Place,End of Meadow Place,55.940238,-3.194640,265,Meadows East,Between Melville Terrace and Sciennes,55.939809,-3.182739
2,2020-01-01 00:21:32.841000+00:00,2020-01-01 01:02:45.899000+00:00,2473,256,St Andrews House,beside Jacobs ladder,55.953164,-3.181682,259,St Andrew Square,North East corner,55.954906,-3.192444


In [15]:

def counted_demand_supply():
    
    #demand processing: dropping columns, 'ended_at' differs 
    find_demand = data2020.drop(['ended_at', 'duration','start_station_id','start_station_description','start_station_latitude','start_station_longitude',
                                     'end_station_id','end_station_description','end_station_latitude','end_station_longitude' ], axis=1)

    #supply processing: dropping colums, 'started_at' differs
    find_supply= data2020.drop(['started_at','duration','start_station_id','start_station_description','start_station_latitude','start_station_longitude','end_station_id',
                                    'end_station_description', 'end_station_latitude','end_station_longitude' ], axis=1)

    #ds ll be used as DateTime parameter & index and the old started_at replaced
    find_demand['ds'] = pd.to_datetime(find_demand['started_at'])


    #ds will be used as DateTime parameter & index and the old ended_at replaced
    find_supply['ds'] =pd.to_datetime(find_supply['ended_at'])


    #the previous started_at and ended_at will be dropped 
    find_demand = find_demand.drop(['started_at'],axis =1)
    find_supply = find_supply.drop(['ended_at'], axis =1)

    #counting demand per hour 
    counted_demand = find_demand.groupby([pd.Grouper(key='ds',freq='H'),find_demand.start_station_name]).size().reset_index(name='count_demand')
    counted_demand = counted_demand.set_index('ds').tz_localize(None)


    #counting supply per hour 
    counted_supply = find_supply.groupby([pd.Grouper(key='ds',freq='H'),find_supply.end_station_name]).size().reset_index(name='count_supply')
    counted_supply = counted_supply.set_index('ds').tz_localize(None)
    
    #saving the outputs in csv 
    counted_demand.to_csv( "counted_demand.csv", encoding='utf-8-sig')
    counted_supply.to_csv( "counted_supply.csv", encoding='utf-8-sig')
    
    return counted_demand, counted_supply

counted_demand_supply()



(                           start_station_name  count_demand
 ds                                                         
 2020-01-01 00:00:00              Meadow Place             1
 2020-01-01 00:00:00          St Andrews House             3
 2020-01-01 00:00:00     Wester Coates Terrace             1
 2020-01-01 00:00:00  Western General Hospital             1
 2020-01-01 01:00:00           Brunswick Place             1
 ...                                       ...           ...
 2020-11-27 22:00:00               Stockbridge             1
 2020-11-27 22:00:00                 Tollcross             1
 2020-11-27 23:00:00            Dundee Terrace             1
 2020-11-27 23:00:00              Meadows East             2
 2020-11-27 23:00:00                 Tollcross             1
 
 [128583 rows x 2 columns],
                               end_station_name  count_supply
 ds                                                          
 2020-01-01 00:00:00              Edinburgh Zoo      